# HW6

In [1]:
https://chatgpt.com/share/672d874d-3f38-8012-8067-197115968e02

# 3


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import norm, uniform
import statsmodels.formula.api as smf

n = 100
beta0 = 2.0
beta1 = 0.5
sigma = 1.0

x = uniform.rvs(size=n, loc=0, scale=10)
epsilon = norm.rvs(size=n, loc=0, scale=sigma)
y = beta0 + beta1 * x + epsilon

df = pd.DataFrame({'x': x, 'Y': y})

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Data'))
x_range = np.linspace(x.min(), x.max(), 100)
y_line = beta0 + beta1 * x_range
fig.add_trace(go.Scatter(x=x_range, y=y_line, mode='lines', name='Theoretical Line', line=dict(dash='dot', color='orange')))
fig.update_layout(title='Theoretical Simple Linear Regression Model', xaxis_title='x', yaxis_title='Y')
fig.show(renderer="png")

model_data_specification = smf.ols("Y ~ x", data=df)
fitted_model = model_data_specification.fit()

print(fitted_model.summary())
print(f"Fitted Parameters: {fitted_model.params}")
print(f"R-squared: {fitted_model.rsquared}")

df['Data'] = 'Data'
fig = px.scatter(df, x='x', y='Y', color='Data', trendline='ols', title='Fitted Simple Linear Regression Model')
fig.show(renderer="png")

x_range = np.array([df['x'].min(), df['x'].max()])
y_line = beta0 + beta1 * x_range
fig.add_scatter(x=x_range, y=y_line, mode='lines', name=f"{beta0} + {beta1} * x", line=dict(dash='dot', color='orange'))
fig.show(renderer="png")


### Key Difference Between the Lines

The **theoretical line** reflects the idealized relationship between $X$ and $Y$ as specified by our chosen model parameters $ \beta_0 $ and $ \beta_1 $, without any random sampling variation. This line doesn’t change across different samples because it represents the model we defined initially.

The **fitted line**, however, is calculated from the observed data, which includes random error terms. As a result, this line is influenced by the particular random sample of errors generated when creating the dataset. If we were to generate a new dataset with different random errors, the fitted line would likely change, even though the theoretical line would remain the same.


# 4


The **fitted values** in `fitted_model.fittedvalues` are calculated using the estimated parameters $ \hat{\beta}_0 $ (intercept) and $ \hat{\beta}_1 $ (slope), which are found in `fitted_model.params`. Each fitted value $ \hat{y}_i $ is given by:

$ \hat{y}_i = \hat{\beta}_0 + \hat{\beta}_1 x_i $

where $ x_i $ is the predictor for each observation.

### Code Example
To manually calculate these fitted values:

```python
# Manually calculate the fitted values using estimated parameters
beta_hat_0 = fitted_model.params[0]
beta_hat_1 = fitted_model.params[1]
fitted_values_manual = beta_hat_0 + beta_hat_1 * df['x']

# Verify they match fitted_model.fittedvalues
print(fitted_values_manual.head())
print(fitted_model.fittedvalues.head())


# 9



In [ ]:
import seaborn as sns
import statsmodels.formula.api as smf

# Load the Old Faithful Geyser dataset
old_faithful = sns.load_dataset('geyser')

# Specify the linear model for duration ~ waiting
model = smf.ols('duration ~ waiting', data=old_faithful)
fitted_model = model.fit()

# Display the summary of the fitted model
fitted_model.summary()




import plotly.express as px

for short_wait_limit in [62, 64, 66]:
    short_wait = old_faithful['waiting'] < short_wait_limit
    model = smf.ols('duration ~ waiting', data=old_faithful[short_wait])
    fitted_model = model.fit()
    
    # Display the summary of the fitted model
    print(f"Summary for short wait limit < {short_wait_limit}")
    print(fitted_model.summary().tables[1])
    
    # Scatter plot with a linear regression trendline
    fig = px.scatter(old_faithful[short_wait], x='waiting', y='duration',
                     title=f"Old Faithful Geyser Eruptions for short wait times (<{short_wait_limit})", 
                     trendline='ols')
    fig.show(renderer="png")  # Use for GitHub and MarkUs submissions


Based on the summaries for each short wait limit, here’s an interpretation of the results:

1. **Short Wait Limit < 62**:
   - **Coefficient for `waiting`**: 0.0069
   - **p-value**: 0.238 (greater than 0.05)
   - **Interpretation**: At a significance level of 0.05, we fail to reject the null hypothesis, suggesting no significant linear association between `waiting` and `duration` for short wait times under 62 minutes.

2. **Short Wait Limit < 64**:
   - **Coefficient for `waiting`**: 0.0114
   - **p-value**: 0.036 (less than 0.05)
   - **Interpretation**: At a significance level of 0.05, we reject the null hypothesis, indicating a statistically significant association between `waiting` and `duration` for short wait times under 64 minutes. This suggests that `waiting` time slightly influences `duration` within this range.

3. **Short Wait Limit < 66**:
   - **Coefficient for `waiting`**: 0.0221
   - **p-value**: 0.000 (well below 0.05)
   - **Interpretation**: At a significance level of 0.05, we strongly reject the null hypothesis, showing clear evidence of a linear association between `waiting` and `duration` for short wait times under 66 minutes. The relationship between `waiting` time and `duration` strengthens as we increase the limit.

### Conclusion
For short wait limits of 64 and 66 minutes, there is evidence of a positive linear relationship between `waiting` and `duration`. However, for wait times under 62 minutes, the relationship is not statistically significant. This indicates that the influence of waiting time on eruption duration becomes more apparent as the wait time threshold increases.

# 11

In [ ]:
import statsmodels.formula.api as smf
import plotly.express as px
from IPython.display import display

# Divide wait times into "short" and "long" based on a threshold of 68 minutes
old_faithful['kind'] = old_faithful['waiting'].apply(lambda x: 'long' if x >= 68 else 'short')

# Fit the Simple Linear Regression model with the indicator variable
indicator_model = smf.ols('duration ~ C(kind, Treatment(reference="short"))', data=old_faithful).fit()

# Display the model summary for evidence against the null hypothesis (no difference between groups)
display(indicator_model.summary().tables[1])

# Visualize the data distribution of duration by kind
fig = px.box(old_faithful, x='kind', y='duration', 
             title='duration ~ kind',
             category_orders={'kind': ['short', 'long']})
fig.show(renderer="png")


Based on the output you provided, here’s a concise interpretation of the results for **Question 11**:

### Interpretation of Results

1. **Intercept (Short Wait Times)**:
   - **Coefficient**: 2.0943
   - This is the estimated mean duration (in minutes) for geyser eruptions following a "short" wait time (less than 68 minutes).
   
2. **C(kind, Treatment(reference="short"))[T.long] (Difference for Long Wait Times)**:
   - **Coefficient**: 2.2036
   - This represents the additional duration (in minutes) for eruptions following a "long" wait time (68 minutes or more), compared to "short" wait times.
   - **Interpretation**: Since the coefficient is positive, "long" wait times are associated with significantly longer eruption durations than "short" wait times, with an average increase of 2.2036 minutes.

3. **Statistical Significance**:
   - **p-value**: 0.000 (for both coefficients), which is well below 0.05.
   - **Conclusion**: We reject the null hypothesis of "no difference between groups on average." There is strong evidence that eruption duration differs significantly between "short" and "long" wait times.

### Summary

This indicator variable model suggests a clear distinction between the durations following "short" and "long" waits, with "long" waits leading to longer eruptions on average by about 2.2 minutes. This finding supports a categorical approach to analyzing the relationship between wait time and eruption duration, contrasting with the previous linear models that used a continuous predictor.